**目标检测的后处理**  

1. IoU计算
2. 非极大值抑制

In [ ]:
import torch
boxes1 = torch.tensor([[2, 2, 6, 6],
                       [8, 8, 12, 12]])

boxes2 = torch.tensor([[3, 3, 7, 7],
                       [10, 10, 14, 14]])

**2D 目标检测**
```python
definition of boxes coordinates: [N, 4]
[x1, y1, x2, y2]
0 <= x1 < x2 and 0 <= y1 < y2
```
也就是左上[0, 0]和右下[a, b]
left_top and right_bottom

In [ ]:
def box_area(boxes):
    """计算bbox面积"""
    return (boxes[:, 2] - boxes[:, 0]) * (boxes[:, 3] - boxes[:, 1])

def _box_inter_union(boxes1, boxes2):
    area1 = box_area(boxes1)
    area2 = box_area(boxes2)

    # 计算相交四边形的左上角和右上角
    lt = torch.max(boxes1[:, None, :2], boxes2[:, :2])  # [N, M, 2]
    rb = torch.min(boxes1[:, None, 2:], boxes2[:, 2:])  # [N, M, 2]

    wh = (rb - lt).clamp(min=0)
    inter = wh[:, :, 0] * wh[:, :, 1]

    union = area1[:, None] + area2 - inter

    return inter, union

def box_iou(boxes1, boxes2):
    inter, union = _box_inter_union(boxes1, boxes2)
    iou = inter / union
    return iou
    

In [ ]:
box_iou(boxes1, boxes2)

**完整实现**

In [ ]:
def NMS(boxes, scores, thresholds):
    x1 = boxes[:,0]
    y1 = boxes[:,1]
    x2 = boxes[:,2]
    y2 = boxes[:,3]
    areas = (x2-x1)*(y2-y1)

    _,order = scores.sort(0,descending=True)
    keep = []
    while order.numel() > 0:
        i = order[0]
        keep.append(i)
        if order.numel() == 1:
            break
        xx1 = x1[order[1:]].clamp(min=x1[i])
        yy1 = y1[order[1:]].clamp(min=y1[i])
        xx2 = x2[order[1:]].clamp(max=x2[i])
        yy2 = y2[order[1:]].clamp(max=y2[i])

        w = (xx2-xx1).clamp(min=0)
        h = (yy2-yy1).clamp(min=0)
        inter = w*h

        ovr = inter/(areas[i] + areas[order[1:]] - inter)
        ids = (ovr<=thresholds).nonzero().squeeze()
        if ids.numel() == 0:
            break
        order = order[ids+1]
    return torch.LongTensor(keep)

**3D 目标检测**

In [ ]:
def batched_nms(boxes, scores, idxs, class_agnossitic):
    """
    Args:
        boxes (torch.Tensor): boxes in shape (N, 4) or (N, 5)
        scores (torch.Tensor): scores in shape (N, ).
        idxs (torch.Tensor): each index value correspond to a bbox cluster,
            and NMS will not be applied between elements of different idxs,
            shape (N, ).
        iou_threshold (float): IoU threshold used for NMS.
        class_agnostic (bool): if true, nms is class agnostic,
            i.e. IoU thresholding happens over all boxes,
            regardless of the predicted class. Defaults to False.
    """